# Obtención de datos

In [3]:
!pip install selenium
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# Obtención de url de productos de la CAV - URL guardadas en CSV

import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup as soup

# Obtener la data de los vinos desde la página de la CAV/tienda
data = []
driver = webdriver.Chrome()

for x in range(50):
    url = f"https://cav.cl/tienda?q=&hPP=20&idx=products&p={x}&fR%5Bfamily.name%5D%5B0%5D=Vinos&is_v=1"
    driver.get(url)
    
    try:
        age_verification_popup = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'modalAge'))
        )
        yes_button = age_verification_popup.find_element(By.XPATH, ".//button[contains(., 'SI')]")
        yes_button.click()
    except (NoSuchElementException, TimeoutException):
        pass

    page_soup = soup(driver.page_source, 'html.parser')
    
    info = []
    for article in page_soup.find_all("article"):
        id = article.attrs.get("data-id")
        sku = article.attrs.get("data-sku")
        name = article.attrs.get("data-name")
        brand = article.attrs.get("data-brand")
        category = article.attrs.get("data-category")
        variant = article.attrs.get("data-variant")
        ref = article.find("a").attrs.get("href")
        info.append([id, sku, name, brand, category, variant, ref])
    
    data.append(info)
    
    # Guardar datos en un archivo CSV después de cada iteración
    df_partial = pd.DataFrame(info, columns=['id', 'sku', 'name', 'brand', 'category', 'variant', 'ref'])
    df_partial.drop_duplicates("id", inplace=True)
    df_partial.to_csv(f"data//raw//vinos_parte_{x}.csv", sep=',', mode='w', index=False)

driver.quit()

# Reshape la data
reshaped_data = [item for sublist in data for item in sublist]

# Crear el DataFrame
df = pd.DataFrame(reshaped_data, columns=['id', 'sku', 'name', 'brand', 'category', 'variant', 'ref'])

# Eliminar los duplicados
df.drop_duplicates("id", inplace=True)

# Guardar CSV de los vinos encontrados
df.to_csv("data//raw//vinos.csv", sep=',', mode='w', index=False)

# Borrar los pack de vinos
df_filtered = df.drop(axis=0, index=df.index[:8])

# Completar la URL en la columna 'ref'
df_filtered['ref'] = 'https://cav.cl/' + df_filtered['ref'].astype(str)

# Guardar CSV solo vinos
df_filtered.to_csv("url_products.csv", sep=',', mode='w', index=False)


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /home/vscode/.cache/selenium/chrome/linux64/119.0.6045.105/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x555ac23695e3 <unknown>
#1 0x555ac202c0b7 <unknown>
#2 0x555ac2062e55 <unknown>
#3 0x555ac205fb81 <unknown>
#4 0x555ac20aa47f <unknown>
#5 0x555ac20a0cc3 <unknown>
#6 0x555ac206c0e4 <unknown>
#7 0x555ac206d0ae <unknown>
#8 0x555ac232fce1 <unknown>
#9 0x555ac2333b7e <unknown>
#10 0x555ac231d4b5 <unknown>
#11 0x555ac23347d6 <unknown>
#12 0x555ac2300dbf <unknown>
#13 0x555ac2357748 <unknown>
#14 0x555ac2357917 <unknown>
#15 0x555ac2368773 <unknown>
#16 0x7faa05e3dea7 start_thread


In [ ]:
#  Obtención de detallada de cada producto desde la URL scrapeada 

import requests
from bs4 import BeautifulSoup
import csv

# Leer las URL desde el archivo CSV
url_list = []

with open('url_products.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        url = row['URL']
        url_list.append(url)

# Crear una lista para almacenar los datos de cada producto
all_product_data = []

for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Acceder a los atributos 'data-*' de la etiqueta 'section'
    product_section = soup.select_one('section.c-product')

    product_id = product_section['data-id']
    product_sku = product_section['data-sku']
    product_name = product_section['data-name']
    product_brand = product_section['data-brand']
    product_category = product_section['data-category']
    product_variant = product_section['data-variant']
    product_price = product_section['data-price']

    # Acceder a la ficha técnica
    wine_table = soup.find('table', class_='table table-striped')

    # Inicializar variables para los campos de la tabla
    wine_data = {
        "Viña": "",
        "Cepa": "",
        "Cosecha": "",
        "Procedencia": "",
        "Línea": "",
        "Formato": "",
        "Tipo": "",
        "Alcohol": "",
        "Mezcla": "",
        "Guarda": "",
        "Maridaje": ""
    }

    if wine_table:
        # Obtener todas las filas de la tabla
        rows = wine_table.find_all('tr')

        for row in rows:
            header = row.find('th')
            data = row.find('td')

            if header and data:
                header_text = header.text.strip()
                data_text = data.text.strip()

                # Asignar los datos a las categorías correspondientes
                if header_text in wine_data:
                    wine_data[header_text] = data_text

    # Acceder al puntaje y calificaciones
    score_section = product_section.select_one('.c-product__score')

    if score_section is not None:
        lacav_score_element = score_section.select_one('.o-score--lacav .o-score__points')
        if lacav_score_element is not None:
            lacav_score = lacav_score_element.text
        else:
            lacav_score = "No score available"
    else:
        lacav_score = "No score available"

    # Descripción
    description = soup.select_one('.c-product__body p').text.strip()

    # Información de precios y stock
    price_info = soup.select_one('.c-product__price')
    socio_price = price_info.select('p.o-text--price-store')[0].text
    normal_price = price_info.select('p.o-text--small')[0].text
    stock = price_info.select('p.o-text--small')[1].text

    # Acceder a la URL de la imagen
    product_image = soup.select_one('div.c-product__thumb img')['src']

    # Crear un diccionario con la información
    product_data = {
        "URL": url,
        "ID del producto": product_id,
        "SKU del producto": product_sku,
        "Nombre del producto": product_name,
        "brand": product_brand,
        "Categoría del producto": product_category,
        "Variante del producto": product_variant,
        "Precio del producto": product_price,
        "Puntaje de La Cav": lacav_score,
        "Descripción": description,
        "Precio Socio": socio_price,
        "Precio Normal": normal_price,
        "Stock": stock,
        "URL de la imagen": product_image,
        "Viña": wine_data["Viña"],
        "Cepa": wine_data["Cepa"],
        "Cosecha": wine_data["Cosecha"],
        "Procedencia": wine_data["Procedencia"],
        "Línea": wine_data["Línea"],
        "Formato": wine_data["Formato"],
        "Tipo": wine_data["Tipo"],
        "Alcohol": wine_data["Alcohol"],
        "Mezcla": wine_data["Mezcla"],
        "Guarda": wine_data["Guarda"],
        "Maridaje": wine_data["Maridaje"]
    }

    all_product_data.append(product_data)

# Resto del código sin cambios

fieldnames = [
    "URL",
    "ID del producto",
    "SKU del producto",
    "Nombre del producto",
    "brand",
    "Categoría del producto",
    "variante",
    "Precio del producto",
    "Puntaje de La Cav",
    "Descripción",
    "Precio Socio",
    "Precio Normal",
    "Stock",
    "URL de la imagen",
    "Viña",
    "Cepa",
    "Cosecha",
    "Procedencia",
    "Línea",
    "Formato",
    "Tipo",
    "Alcohol",
    "Mezcla",
    "Guarda",
    "Maridaje",
    "Cuerpo",
    "Frescor",
    "Dulzura",
    "Astringencia"
]

# Guardar los datos en un archivo CSV
csv_filename = "productos.csv"

with open(csv_filename, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for product_data in all_product_data:
        writer.writerow(product_data)


# Almacenamiento y tratamiento de conjunto de datos 

In [1]:
import pandas as pd

productos = pd.read_csv('../raw/scrap_cav/productos.csv') 
meridaje = pd.read_csv('../raw/meridaje.csv')
ranking = pd.read_csv('../raw/brand_shopper_rank.csv') 

hello


In [ ]:
import sqlite3

conn = sqlite3.connect("../wine_db.db")
productos.to_sql("productos", conn, if_exists="replace", index=False)
meridaje.to_sql("meridaje", conn, if_exists="replace", index=False)
ranking.to_sql("ranking", conn, if_exists="replace", index=False)

In [ ]:
query = """
SELECT *
FROM productos
JOIN meridaje
ON productos.variante = meridaje.variante
LEFT JOIN ranking
ON productos.brand = ranking.brand
;
"""

df = pd.read_sql_query(query, conn)

conn.close()

df.info()

In [ ]:
df = df.loc[:, ~df.columns.duplicated()]

df.columns

# Análisis descriptivo